In [1]:
import pandas as pd
import numpy as np
import glob

In [3]:
# 엑셀 파일 저장 관련 개인 함수
# 참고
# 생성 결과를 클립보드로 복사
# merge_df_계열오류_일전_특전.to_clipboard(index=False)
# 기존 엑셀 파일 저장 명령
# save_path = file_path+"등록대상자생성결과(재학생 + 휴학생)_책정등록금확인"+file_format
# merge_register_table.to_excel(save_path, index=True)
def my_excel_save(save_df, save_name, save_path):
    save_format = ".xlsx" # 저장 파일 확장자
    save_file_path = save_path+save_name+save_format # 저장 파일 경로 및 파일 명
    save_df.to_excel(save_file_path, index=True) # 저장


In [4]:
# 파일 경로 및 확장자 지정 .xlsx 와 .csv에 따라 진행
file_format = ".xlsx" # .csv
file_path = "D:\C-myPyExcel\재학생 등록자료 검토\\" # 경로 입력
# file_list = glob.glob(f"{file_path}/*{file_format}")
# file_list

In [5]:
# 학과별 등록금 책정 자료 읽어오기
df_register_table = pd.read_excel(
                                "230626_(자료) 종헙정보시스템 학사행정_등록_기준정보_기준정보 중 등록금 책정 관리(전공별) - 계약학과 수정 후 검토용"+file_format,
                                sheet_name="피벗결과"
                                           )

df_register_table

,학부(과),단과대학,계열구분,사용여부,입학금,입학금 실비용,수업료,총합계
0,가정교육과,사범대학,이학계열,Checked,0,112400,3211000,3323400
1,간호학과,건강보건대학,이학계열,Checked,0,112400,3211000,3323400
2,건축학부,공과대학,공학계열,Checked,0,112400,3750000,3862400
3,경영학과,경영대학,"인문,사회",Checked,0,112400,2200000,2312400
4,경영학부,경영대학,"인문,사회",Checked,0,112400,2722000,2834400
...,...,...,...,...,...,...,...,...
69,토목공학과,공과대학,공학계열,Checked,0,112400,3750000,3862400
70,산업디자인학과,공과대학,예능,Checked,0,112400,3869000,3981400
71,식품영양생명학부,건강보건대학,이학계열,Checked,0,112400,3211000,3323400
72,국제무역물류학과,경영대학,"인문,사회",Checked,0,112400,2722000,2834400


In [6]:
# 등록대상자생성결과(재학생) 파일 읽어오기
df_register_in_shool = pd.read_excel(
                                "230203_(자료) 종합정보 등록대상자생성(재학생) 등록발부자 11196명"+file_format,
                                usecols=["발부일자", "등록구분", "학부(과)", "학년", "학번", "입학금", "수업료", "등록금", "장학1", "장학2", "입학감면", "수업감면", "전체감면", "수납금액", "가상계좌", "수납여부"],
                                sheet_name="Sheet",
                                skipfooter=1
                                           )

# 등록구분에 따라 등록구분 그룹화 약명 추가
map_dictionary ={
                    "복학전액등록":"복학전액",
                    "복학제로등록":"복학제로",
                    "재수졸업인증수강등록":"졸업인증수강",
                    "재학생등록":"재학",
                    "편입학등록":"편입",
                    "신입학등록":"신입",
                    "학사학위취득유예등록":"학사학위취득유예",
                    "재수재입학":"재입학", "재수재입학반액등록":"재입학", "재수재입학전액등록":"재입학", "재입학전액등록":"재입학",
                    "재수1/3":"수업연한초과", "재수1/6":"수업연한초과", "재수반액등록":"수업연한초과", "재수복학1/3":"수업연한초과", "재수복학1/6":"수업연한초과", "재수복학반액":"수업연한초과",
                    "재수복학전액":"수업연한초과", "재수복학제로":"수업연한초과", "재수전액등록":"수업연한초과"} 
df_register_in_shool["등록구분(그룹)"] = df_register_in_shool["등록구분"].map(map_dictionary) 

# nan 값 없애기
df_register_in_shool = df_register_in_shool.fillna("")

# 등록구분 추가
df_register_in_shool["등록구분(재휴)"] = "재학"

# 데이터 타입 변경
df_register_in_shool["가상계좌"] = pd.to_numeric(df_register_in_shool["가상계좌"], errors="coerce")

In [7]:
# 등록대상자생성결과(휴학생) 파일 읽어오기
df_register_leave = pd.read_excel(
                                "230203_(자료) 종합정보 등록대상자생성(휴학생) - 실행 후 등록발부자 2248명-가상계좌 부여 후"+file_format,
                                usecols=["발부일자", "등록구분", "학부(과)", "학년", "학번", "입학금", "수업료", "등록금", "장학", "입학감면", "수업감면", "감면합계", "가상계좌"],
                                sheet_name="Sheet",
                                skipfooter=1
                                           )

# 등록구분에 따라 등록구분 그룹화 약명 추가
map_dictionary ={
                    "복학전액등록":"복학전액",
                    "복학제로등록":"복학제로",
                    "재수졸업인증수강등록":"졸업인증수강",
                    "재학생등록":"재학",
                    "편입학등록":"편입",
                    "신입학등록":"신입",
                    "학사학위취득유예등록":"학사학위취득유예",
                    "재수재입학":"재입학", "재수재입학반액등록":"재입학", "재수재입학전액등록":"재입학", "재입학전액등록":"재입학", "재수재입학1/3":"재입학",
                    "재수1/3":"수업연한초과", "재수1/6":"수업연한초과", "재수반액등록":"수업연한초과", "재수복학1/3":"수업연한초과", "재수복학1/6":"수업연한초과", "재수복학반액":"수업연한초과",
                    "재수복학전액":"수업연한초과", "재수복학제로":"수업연한초과", "재수전액등록":"수업연한초과"} 
df_register_leave["등록구분(그룹)"] = df_register_leave["등록구분"].map(map_dictionary) 


df_register_leave.rename(
                   columns={
                               "감면합계":"전체감면",
                           }
                            , inplace=True
                            )

# 등록구분 추가
df_register_leave["등록구분(재휴)"] = "휴학"


# nan 값 없애기
df_register_leave = df_register_leave.fillna("")

In [8]:
# 위 등록대상자생성결과(재학생 + 휴학생)를 하나의 dataframe 생성됨 -아래로 병합: concat - 재학과 휴학에 중복된 자료도 각각 검증
co_register_등록금 = pd.concat([df_register_in_shool, df_register_leave], ignore_index=True)

co_register_등록금 = co_register_등록금[[
                                            "발부일자",
                                            "등록구분",   
                                            "학부(과)",
                                            "학년",
                                            "학번",
                                            "입학금",
                                            "수업료",
                                            "등록금",
                                            "입학감면",
                                            "수업감면",
                                            "전체감면",
                                            "등록구분(그룹)",
                                            "등록구분(재휴)"
                                        ]]

co_register_등록금["입학금+수업료"] = co_register_등록금["입학금"] + co_register_등록금["수업료"]
co_register_등록금["입학감면+수업감면"] = co_register_등록금["입학감면"] + co_register_등록금["수업감면"]

# 책정 등록금 = 등록금_재학 + 전체감면_재학
co_register_등록금["책정 등록금"] = co_register_등록금["등록금"] + co_register_등록금["전체감면"]


# 자료 비교로 이상여부 확인
co_register_등록금.loc[co_register_등록금["등록금"] == co_register_등록금["입학금+수업료"], "등록금비교(등록금=입학금+수업료)"] = "일치"
co_register_등록금.loc[co_register_등록금["등록금"] != co_register_등록금["입학금+수업료"], "등록금비교(등록금=입학금+수업료)"] = "불일치"

co_register_등록금.loc[co_register_등록금["전체감면"] == co_register_등록금["입학감면+수업감면"], "전체감면비교(전체감면=입학감면+수업감면)"] = "일치"
co_register_등록금.loc[co_register_등록금["전체감면"] != co_register_등록금["입학감면+수업감면"], "전체감면비교(전체감면=입학감면+수업감면)"] = "불일치"


# 위 등록대상자생성결과(재학생 + 휴학생)에서 가상계좌 기준으로 합쳐진 중복 값 생성
merge_register_table = pd.merge(co_register_등록금, df_register_table, how="outer", left_on="학부(과)", right_on="학부(과)", suffixes=('_책정', '_table'))


# 엑셀 파일 저장, def my_excel_save(저장 데이터프레임, 저장 파일명, 저장 경료):
my_excel_save(merge_register_table, "등록대상자생성결과(재학생 + 휴학생)_책정등록금확인", "D:\C-myPyExcel\재학생 등록자료 검토\\")

In [86]:
# 일반 전과 합격자 파일 읽어오기
df_일전 = pd.read_excel(
                        "붙임8-1 2023-1 전과합격자(학과용)"+file_format,
                        usecols=["학번", "학년", "학적\n상태", "전출학부(과)", "전입학(부)과", "전출입\n학년"],
                        sheet_name="Sheet"
                        )

# 명칭 변경
df_일전.rename(
                   columns={
                                "학적\n상태":"학적상태",
                                "학년":"전출학년",
                                "전출입\n학년":"전입학년",
                                "전출학부(과)":"전출학과",
                                "전입학(부)과":"전입학과"
                           },
                   inplace=True
                )

# 구분 추가
df_일전["전과구분"] = "일반"

df_일전.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   학번      59 non-null     int64 
 1   전출학년    59 non-null     int64 
 2   학적상태    59 non-null     object
 3   전출학과    59 non-null     object
 4   전입학과    59 non-null     object
 5   전입학년    59 non-null     int64 
 6   전과구분    59 non-null     object
dtypes: int64(3), object(4)
memory usage: 3.4+ KB


In [87]:
# 특별 전과 합격자 파일 읽어오기
df_특전 = pd.read_excel(
                        "붙임8-2 2023-1 특별전과 합격자(학과용)"+file_format,
                        usecols=["학번", "학년", "학적\n상태", "현 소속", "전부(과) 소속", "전부(과)\n학년", "복학예정연도", "복학예정학기"],
                        skiprows=1,
                        sheet_name="Sheet"
                        )

# 명칭 변경
df_특전.rename(
                   columns={
                               "학적\n상태":"학적상태",
                               "학년":"전출학년",
                               "현 소속":"전출학과",
                               "전부(과) 소속":"전입학과",
                               "전부(과)\n학년":"전입학년"
                           },
                   inplace=True
                )

# 구분 추가
df_특전["전과구분"] = "특별"

# nan 값 없애기
df_특전["복학예정연도"] = df_특전["복학예정연도"].fillna("재학")
df_특전["복학예정학기"] = df_특전["복학예정학기"].fillna("재학")


df_특전.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   학번      19 non-null     int64 
 1   전출학년    19 non-null     int64 
 2   학적상태    19 non-null     object
 3   전출학과    19 non-null     object
 4   전입학과    19 non-null     object
 5   전입학년    19 non-null     int64 
 6   복학예정연도  19 non-null     object
 7   복학예정학기  19 non-null     object
 8   전과구분    19 non-null     object
dtypes: int64(3), object(6)
memory usage: 1.5+ KB


In [88]:
# 위 일전 + 특전 에서 학번 기준으로 합쳐진 중복 값 생성
co_일전_특전 = pd.concat([df_일전, df_특전], ignore_index=True)

co_일전_특전

,학번,전출학년,학적상태,전출학과,전입학과,전입학년,전과구분,복학예정연도,복학예정학기
0,2022110250,1,재학,스포츠과학과,건설시스템공학과,2,일반,NaN,NaN
1,2019111867,1,재학,경찰학부,건설시스템공학과,2,일반,NaN,NaN
2,2018112860,3,휴학,조선해양시스템공학과,건설시스템공학과,3,일반,NaN,NaN
3,2019112765,2,재학,환경에너지공학과,건축학부,3,일반,NaN,NaN
4,2018112863,3,재학,조선해양시스템공학과,건축학부,4,일반,NaN,NaN
...,...,...,...,...,...,...,...,...,...
73,2021110051,2,휴학,중국학과,미디어영상학과,2,특별,2023.0,1.0
74,2020110028,2,휴학,한국어문학과,건설시스템공학과,2,특별,2023.0,1.0
75,2022111573,1,재학,환경에너지공학과,전자공학과,2,특별,재학,재학
76,2020112611,2,휴학,환경에너지공학과,컴퓨터공학부,2,특별,2023.0,1.0


In [93]:
# 2 위 등록대상자생성결과(재학생 + 휴학생)에서 가상계좌 기준으로 합쳐진 중복 값 생성
merge_register_table = pd.merge(co_register_등록금, df_register_table, how="outer", left_on="학부(과)", right_on="학부(과)", suffixes=('_책정', '_table'))

# 위 자료에서 등록자료 붙이기
merge_일전_특전_등록자료 = pd.merge(co_일전_특전, merge_register_table, how="inner", left_on="학번", right_on="학번", suffixes=('_전과', '_등록'))

merge_일전_특전_등록자료 = merge_일전_특전_등록자료.drop(["발부일자", "학부(과)", "학년", "입학금_책정", "수업료_책정", "입학감면", "수업감면", "입학금+수업료", "입학감면+수업감면",
                                                          "책정 등록금", "등록금비교(등록금=입학금+수업료)", "전체감면비교(전체감면=입학감면+수업감면)", "사용여부"]
                                                         , axis=1)

# 엑셀 파일 저장, def my_excel_save(저장 데이터프레임, 저장 파일명, 저장 경료):
my_excel_save(merge_일전_특전_등록자료, "merge_일전_특전_등록자료", "D:\C-myPyExcel\재학생 등록자료 검토\\")

merge_일전_특전_등록자료.head(5)

,학번,전출학년,학적상태,전출학과,전입학과,전입학년,전과구분,복학예정연도,복학예정학기,등록구분,등록금,전체감면,등록구분(그룹),등록구분(재휴),단과대학,계열구분,입학금_table,입학금 실비용,수업료_table,총합계
0,2022110250,1,재학,스포츠과학과,건설시스템공학과,2,일반,NaN,NaN,재학생등록,3750000,0,재학,재학,공과대학,공학계열,0,112400,3750000,3862400
1,2019111867,1,재학,경찰학부,건설시스템공학과,2,일반,NaN,NaN,재학생등록,1250000,2500000,재학,재학,공과대학,공학계열,0,112400,3750000,3862400
2,2018112860,3,휴학,조선해양시스템공학과,건설시스템공학과,3,일반,NaN,NaN,복학전액등록,3750000,0,복학전액,재학,공과대학,공학계열,0,112400,3750000,3862400
3,2019112765,2,재학,환경에너지공학과,건축학부,3,일반,NaN,NaN,재학생등록,1050000,2700000,재학,재학,공과대학,공학계열,0,112400,3750000,3862400
4,2018112863,3,재학,조선해양시스템공학과,건축학부,4,일반,NaN,NaN,재학생등록,3000000,750000,재학,재학,공과대학,공학계열,0,112400,3750000,3862400


## 과/학년/전공오류자 탭에서 등록학과<>학적학과 오류 파일에 학사관리팀 전과 결과 합치기

In [94]:
# 등록학과 학적학과 오류 파일 읽어오기
df_학과오류 = pd.read_excel(
                        "230203_(자료) 종합정보 등록검색자료출력 등록학과와 학적학과 오류자 LIST.xlsx",
                        usecols=["연번", "학번", "등록단과대학", "등록학과", "등록학년", "학적단과대학", "학적학과", "학적학년", "비고", "최종학적변동"],
                        sheet_name="Sheet"
                        )

df_학과오류.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      158 non-null    int64 
 1   학번      158 non-null    int64 
 2   등록단과대학  158 non-null    object
 3   등록학과    158 non-null    object
 4   등록학년    158 non-null    int64 
 5   학적단과대학  158 non-null    object
 6   학적학과    158 non-null    object
 7   학적학년    158 non-null    int64 
 8   비고      158 non-null    object
 9   최종학적변동  158 non-null    object
dtypes: int64(4), object(6)
memory usage: 12.5+ KB


In [95]:
# 학과 오류 파일에 전과 자료 합치기 학번를 기준으로 하나의 dataframe 생성됨
merge_df_학과오류_일전_특전 = pd.merge(df_학과오류, merge_일전_특전_등록자료, left_on="학번", right_on="학번", how="left")

# 생성 결과를 클립보드로 복사
# merge_df_학과오류_일전_특전.to_clipboard(index=False)

# 엑셀 파일 저장, def my_excel_save(저장 데이터프레임, 저장 파일명, 저장 경료):
my_excel_save(merge_df_학과오류_일전_특전, "merge_df_학과오류_일전_특전", "D:\C-myPyExcel\재학생 등록자료 검토\\")


merge_df_학과오류_일전_특전.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 29 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   연번         158 non-null    int64  
 1   학번         158 non-null    int64  
 2   등록단과대학     158 non-null    object 
 3   등록학과       158 non-null    object 
 4   등록학년       158 non-null    int64  
 5   학적단과대학     158 non-null    object 
 6   학적학과       158 non-null    object 
 7   학적학년       158 non-null    int64  
 8   비고         158 non-null    object 
 9   최종학적변동     158 non-null    object 
 10  전출학년       6 non-null      float64
 11  학적상태       6 non-null      object 
 12  전출학과       6 non-null      object 
 13  전입학과       6 non-null      object 
 14  전입학년       6 non-null      float64
 15  전과구분       6 non-null      object 
 16  복학예정연도     4 non-null      object 
 17  복학예정학기     4 non-null      object 
 18  등록구분       6 non-null      object 
 19  등록금        6 non-null      float64
 20  전체감면      

## 과/학년/전공오류자 탭에서 등록전공<>학적전공 오류 파일에 학사관리팀 전과 결과 합치기

In [96]:
# 등록전공 학적전공 오류 파일 읽어오기
df_전공오류 = pd.read_excel(
                        "230203_(자료) 종합정보 등록검색자료출력 등록전공과 학적전공 오류자 LIST.xlsx",
                        usecols=["연번", "학번", "등록학과", "등록전공", "학적전공", "최종학적변동"],
                        sheet_name="Sheet"
                        )

df_전공오류.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      165 non-null    int64 
 1   학번      165 non-null    int64 
 2   등록학과    165 non-null    object
 3   등록전공    165 non-null    object
 4   학적전공    165 non-null    object
 5   최종학적변동  165 non-null    object
dtypes: int64(2), object(4)
memory usage: 7.9+ KB


In [97]:
# 전공 오류 파일에 전과 자료 합치기 학번를 기준으로 하나의 dataframe 생성됨
merge_df_전공오류_일전_특전 = pd.merge(df_전공오류, merge_일전_특전_등록자료, left_on="학번", right_on="학번", how="left")

# 생성 결과를 클립보드로 복사
merge_df_전공오류_일전_특전.to_clipboard(index=False)

merge_df_전공오류_일전_특전.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 0 to 164
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   연번         165 non-null    int64  
 1   학번         165 non-null    int64  
 2   등록학과       165 non-null    object 
 3   등록전공       165 non-null    object 
 4   학적전공       165 non-null    object 
 5   최종학적변동     165 non-null    object 
 6   전출학년       6 non-null      float64
 7   학적상태       6 non-null      object 
 8   전출학과       6 non-null      object 
 9   전입학과       6 non-null      object 
 10  전입학년       6 non-null      float64
 11  전과구분       6 non-null      object 
 12  복학예정연도     4 non-null      object 
 13  복학예정학기     4 non-null      object 
 14  등록구분       6 non-null      object 
 15  등록금        6 non-null      float64
 16  전체감면       6 non-null      float64
 17  등록구분(그룹)   6 non-null      object 
 18  등록구분(재휴)   6 non-null      object 
 19  단과대학       6 non-null      object 
 20  계열구분      

## 과/학년/전공오류자 탭에서 등록계열<>학적계열 오류 파일에 학사관리팀 전과 결과 합치기

In [98]:
# 등록계열 학적계열 오류 파일 읽어오기
df_계열오류 = pd.read_excel(
                        "230203_(자료) 종합정보 등록검색자료출력 등록계열과 학적계열 오류자 LIST"+file_format,
                        usecols=["연번", "학번", "등록학과", "등록계열", "학적학과", "학적계열", "최종학적변동"],
                        sheet_name="Sheet"
                        )

df_계열오류.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      4 non-null      int64 
 1   학번      4 non-null      int64 
 2   등록학과    4 non-null      object
 3   등록계열    4 non-null      object
 4   학적학과    4 non-null      object
 5   학적계열    4 non-null      object
 6   최종학적변동  4 non-null      object
dtypes: int64(2), object(5)
memory usage: 352.0+ bytes


In [99]:
# 계열 오류 파일에 전과 자료 합치기 학번를 기준으로 하나의 dataframe 생성됨
merge_df_계열오류_일전_특전 = pd.merge(df_계열오류, merge_일전_특전_등록자료, left_on="학번", right_on="학번", how="left")

# 생성 결과를 클립보드로 복사
# merge_df_계열오류_일전_특전.to_clipboard(index=False)

# 엑셀 파일 저장, def my_excel_save(저장 데이터프레임, 저장 파일명, 저장 경료):
my_excel_save(merge_df_계열오류_일전_특전, "merge_df_계열오류_일전_특전", "D:\C-myPyExcel\재학생 등록자료 검토\\")

merge_df_계열오류_일전_특전.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   연번         4 non-null      int64 
 1   학번         4 non-null      int64 
 2   등록학과       4 non-null      object
 3   등록계열       4 non-null      object
 4   학적학과       4 non-null      object
 5   학적계열       4 non-null      object
 6   최종학적변동     4 non-null      object
 7   전출학년       4 non-null      int64 
 8   학적상태       4 non-null      object
 9   전출학과       4 non-null      object
 10  전입학과       4 non-null      object
 11  전입학년       4 non-null      int64 
 12  전과구분       4 non-null      object
 13  복학예정연도     2 non-null      object
 14  복학예정학기     2 non-null      object
 15  등록구분       4 non-null      object
 16  등록금        4 non-null      int64 
 17  전체감면       4 non-null      int64 
 18  등록구분(그룹)   4 non-null      object
 19  등록구분(재휴)   4 non-null      object
 20  단과대학       4 non-null      object
 2

## 장학 초과 부여자 확인

In [105]:
# 아래 저장된 엑셀 파일을 엑셀 피벗으로 분석
# 엑셀 파일 저장
# save_path = file_path+"등록대상자생성결과(재학생 + 휴학생)_책정등록금확인"+file_format
# merge_register_table.to_excel(save_path, index=True)

## 졸업대상자 중 등록금납입통지서 발급자가 있는지 확인 - 학사관리팀 졸업예정자 명단과 등록대상자 비교

In [108]:
# 학사관리팀 졸업자 명단 읽어오기
df_affairs_finish = pd.read_excel(
                                "붙임7 졸업확정자명단(23.1.30.)"+file_format,
                                sheet_name="Sheet"
                                           )

df_affairs_finish

,대학,학부(과),전공,학번,성명,전공학위
0,NaN,NaN,NaN,2023111810,NaN,NaN
1,사범대학,교육학과,교육학,2017110722,이민기,교육학사
2,사범대학,교육학과,교육학,2017110734,성열완,교육학사
3,사범대학,교육학과,교육학,2017110738,남찬우,교육학사
4,사범대학,교육학과,교육학,2019110805,배천수,교육학사
...,...,...,...,...,...,...
1967,AI·SW융합대학,정보통신AI공학과,정보통신AI공학,2019112608,정예린,공학사
1968,AI·SW융합대학,정보통신AI공학과,정보통신AI공학,2019112611,김채은,공학사
1969,AI·SW융합대학,정보통신AI공학과,정보통신AI공학,2019112618,방혜경,공학사
1970,AI·SW융합대학,정보통신AI공학과,정보통신AI공학,2019113298,김재민,공학사


In [109]:
# 등록대상자생성결과(재학생 + 휴학생)에서 가상계좌 기준으로 합쳐진 Dataframe과 비교 - merge_register_table

# 계열 오류 파일에 전과 자료 합치기 학번를 기준으로 하나의 dataframe 생성됨
merge_df_등록대상자_졸업대상자_비교 = pd.merge(merge_register_table, df_affairs_finish, left_on="학번", right_on="학번", how="inner")

# 엑셀 파일 저장, def my_excel_save(저장 데이터프레임, 저장 파일명, 저장 경료):
my_excel_save(merge_df_등록대상자_졸업대상자_비교, "merge_df_등록대상자와 졸업대상자 비교", "D:\C-myPyExcel\재학생 등록자료 검토\\")

merge_df_등록대상자_졸업대상자_비교

,발부일자,등록구분,학부(과)_x,학년,학번,입학금_책정,수업료_책정,등록금,입학감면,수업감면,...,사용여부,입학금_table,입학금 실비용,수업료_table,총합계,대학,학부(과)_y,전공,성명,전공학위
0,2023-03-02 08:52:24,신입학등록,자유전공학부,1,2023111810,0,3323400,3323400,0,0,...,Checked,0,112400,3211000,3323400,NaN,NaN,NaN,NaN,NaN
1,2023-03-02 08:55:25,신입학등록,자유전공학부,1,2017110722,0,99999,99999,0,0,...,Checked,0,112400,3211000,3323400,사범대학,교육학과,교육학,이민기,교육학사


## 재입학자 중 수업연한초과자의 등록금 확인

In [115]:
# 학사관리팀 재입학자 명단 읽어오기
df_affairs_readmission = pd.read_excel(
                                "붙임2 2023-1 재입학 허가자(1차) 명단"+file_format,
                                sheet_name="최종본",
                                usecols=["학번", "학년\n학기차", "변경후 소속", "장학지급대상", "수업연한초과자", "비고"]
                                           )

df_affairs_readmission

,학번,학년\n학기차,변경후 소속,장학지급대상,수업연한초과자,비고
0,2003160128,4,공공인재대학 법학과 법학,반액감면대상,수업연한초과자,NaN
1,2017111396,2,경영대학 경영학부 회계학,반액감면대상,NaN,NaN
2,2014112104,2,공공인재대학 경찰학부 경찰학,반액감면대상,NaN,NaN
3,2011112215,3,공공인재대학 경찰학부 경찰학,반액감면대상,NaN,NaN
4,2009112244,4,공공인재대학 행정학과 행정학,반액감면대상,수업연한초과자,NaN
5,2013113217,4,공과대학 환경에너지공학과 환경에너지공학,반액감면대상,NaN,NaN
6,2008113460,4,경영대학 경영학부 경영학,반액감면대상,수업연한초과자,NaN
7,2015111991,4,공공인재대학 경찰학부 경찰학,반액감면대상,NaN,NaN
8,2017112921,2,공과대학 산업디자인학과 산업디자인학,반액감면대상,NaN,NaN
9,2018111296,2,경영대학 경영학부 경영학,반액감면대상,NaN,NaN


In [116]:
# 등록대상자생성결과(재학생 + 휴학생)에서 가상계좌 기준으로 합쳐진 Dataframe과 비교 - merge_register_table

# 계열 오류 파일에 전과 자료 합치기 학번를 기준으로 하나의 dataframe 생성됨
merge_df_등록대상자_재입학자_비교 = pd.merge(merge_register_table, df_affairs_readmission, left_on="학번", right_on="학번", how="inner")



# 엑셀 파일 저장, def my_excel_save(저장 데이터프레임, 저장 파일명, 저장 경료):
my_excel_save(merge_df_등록대상자_재입학자_비교, "merge_df_등록대상자와 재입학자 비교", "D:\C-myPyExcel\재학생 등록자료 검토\\")

merge_df_등록대상자_재입학자_비교

,발부일자,등록구분,학부(과),학년,학번,입학금_책정,수업료_책정,등록금,입학감면,수업감면,...,사용여부,입학금_table,입학금 실비용,수업료_table,총합계,학년\n학기차,변경후 소속,장학지급대상,수업연한초과자,비고
0,2023-02-03 11:30:17,재입학전액등록,행정학과,3,2015111950,0,1304800,1304800,0,1529600,...,Checked,0,112400,2722000,2834400,3,공공인재대학 행정학과 행정학,반액감면대상,NaN,NaN
1,2023-02-03 11:30:17,재수재입학전액등록,행정학과,4,2009112244,0,1304800,1304800,0,1529600,...,Checked,0,112400,2722000,2834400,4,공공인재대학 행정학과 행정학,반액감면대상,수업연한초과자,NaN
2,2023-02-03 11:30:17,재입학전액등록,경찰학부,2,2014112104,0,1304800,1304800,0,1529600,...,Checked,0,112400,2722000,2834400,2,공공인재대학 경찰학부 경찰학,반액감면대상,NaN,NaN
3,2023-02-03 11:30:17,재입학전액등록,경찰학부,4,2011112215,0,1304800,1304800,0,1529600,...,Checked,0,112400,2722000,2834400,3,공공인재대학 경찰학부 경찰학,반액감면대상,NaN,NaN
4,2023-02-03 11:30:17,재입학전액등록,경찰학부,4,2015111991,0,1304800,1304800,0,1529600,...,Checked,0,112400,2722000,2834400,4,공공인재대학 경찰학부 경찰학,반액감면대상,NaN,NaN
5,2023-02-03 11:30:17,재입학전액등록,경찰학부,4,2016111939,0,1304800,1304800,0,1529600,...,Checked,0,112400,2722000,2834400,4,공공인재대학 경찰학부 경찰학,반액감면대상,NaN,NaN
6,2023-02-03 11:30:17,재입학전액등록,심리학과,2,2020110541,0,1304800,1304800,0,1529600,...,Checked,0,112400,2722000,2834400,1,인문사회대학 심리학과 심리학,반액감면대상,NaN,심리학과\n특별전과 신청
7,2023-02-03 11:30:17,재입학전액등록,미디어영상학과,2,2021111297,0,1427300,1427300,0,1652100,...,Checked,0,112400,2967000,3079400,2,공공인재대학 미디어영상학과 미디어영상학,반액감면대상,NaN,NaN
8,2023-02-03 11:30:17,재수재입학전액등록,문화콘텐츠학과,4,2016110241,0,1549300,1549300,0,1774100,...,Checked,0,112400,3211000,3323400,4,인문사회대학 문화콘텐츠학과 문화콘텐츠학,반액감면대상,수업연한초과자,NaN
9,2023-02-03 11:30:17,재수재입학반액등록,법학과,4,2003160128,0,596200,596200,0,821000,...,Checked,0,112400,2722000,2834400,4,공공인재대학 법학과 법학,반액감면대상,수업연한초과자,NaN


In [4]:
# 기납자 파일 읽어오기
df_기납 = pd.read_excel(
                        "230203_(자료) 종합정보 기납대상자 생성 결과 723명.xlsx",
                        sheet_name="Sheet"
                        )

# 등록구분에 nan 값이 존재할 경우 "미등록"로 표시
df_기납["등록구분"] = df_기납["등록구분"].fillna("미등록")

df_기납.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   연번          724 non-null    int64         
 1   학번          724 non-null    int64         
 2   성명          724 non-null    object        
 3   전공          724 non-null    object        
 4   기납시_학적변동내역  40 non-null     object        
 5   기납시_학적변동일자  724 non-null    datetime64[ns]
 6   기납구분        724 non-null    object        
 7   기납년도        724 non-null    int64         
 8   기납학기        724 non-null    int64         
 9   등록구분        724 non-null    object        
 10  등록여부        0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 62.3+ KB


In [5]:
# 휴학 후 환불자 파일 읽어오기
df_휴학환불 = pd.read_excel(
                        "230203_(자료) 종합정보 등록 후 휴학환불자 18명(21년 3월 1일부터 23년 2월 3일까지).xlsx",
                        sheet_name="가공",
                        usecols=["연번", "학번", "성명", "학과명", "학년", "구분", "입학금", "수업료", "합계", "처리일자", "년도/학기", "사유"]
                        )

df_휴학환불.rename(
                   columns={"연번":"휴학환불_연번",
                            "학번":"휴학환불_학번",
                            "성명":"휴학환불_성명",
                            "학과명":"휴학환불_학과명",
                            "학년":"휴학환불_학년",
                            "구분":"휴학환불_구분",
                            "입학금":"휴학환불_입학금",
                            "수업료":"휴학환불_수업료",
                            "합계":"휴학환불_합계",
                            "처리일자":"휴학환불_처리일자",
                            "년도/학기":"휴학환불_년도/학기",
                            "사유":"휴학환불_사유"}
                            , inplace=True
                            )

df_휴학환불.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   휴학환불_연번     19 non-null     int64         
 1   휴학환불_학번     19 non-null     int64         
 2   휴학환불_성명     19 non-null     object        
 3   휴학환불_학과명    19 non-null     object        
 4   휴학환불_학년     19 non-null     int64         
 5   휴학환불_구분     19 non-null     object        
 6   휴학환불_입학금    19 non-null     int64         
 7   휴학환불_수업료    19 non-null     int64         
 8   휴학환불_합계     19 non-null     int64         
 9   휴학환불_처리일자   19 non-null     datetime64[ns]
 10  휴학환불_년도/학기  19 non-null     object        
 11  휴학환불_사유     19 non-null     object        
dtypes: datetime64[ns](1), int64(6), object(5)
memory usage: 1.9+ KB


In [6]:
# 기납자 파일의 수험번호를 기준으로 하나의 dataframe 생성됨
merge_df_기납휴학비교 = pd.merge(df_기납, df_휴학환불,left_on="학번", right_on="휴학환불_학번", how="outer")

# merge 후 등록구분에 nan 값이 존재할 경우 "휴학환불"로 표시
merge_df_기납휴학비교["등록구분"] = merge_df_기납휴학비교["등록구분"].fillna("휴학환불")

# merge 후 기납구분에 nan 값이 존재할 경우 "휴학환불"로 표시
merge_df_기납휴학비교["기납구분"] = merge_df_기납휴학비교["기납구분"].fillna("휴학환불")


# 인덱스 초기화
merge_df_기납휴학비교.reset_index(drop=True, inplace=True)

merge_df_기납휴학비교.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   연번          724 non-null    float64       
 1   학번          724 non-null    float64       
 2   성명          724 non-null    object        
 3   전공          724 non-null    object        
 4   기납시_학적변동내역  40 non-null     object        
 5   기납시_학적변동일자  724 non-null    datetime64[ns]
 6   기납구분        742 non-null    object        
 7   기납년도        724 non-null    float64       
 8   기납학기        724 non-null    float64       
 9   등록구분        742 non-null    object        
 10  등록여부        0 non-null      float64       
 11  휴학환불_연번     19 non-null     float64       
 12  휴학환불_학번     19 non-null     float64       
 13  휴학환불_성명     19 non-null     object        
 14  휴학환불_학과명    19 non-null     object        
 15  휴학환불_학년     19 non-null     float64       
 16  휴학환불_구분     19 non-null   

In [7]:
# 동일 학번 있는지 비교
merge_df_기납휴학비교.loc[merge_df_기납휴학비교["학번"] == merge_df_기납휴학비교["휴학환불_학번"], "학번비교"] = "일치"
merge_df_기납휴학비교.loc[merge_df_기납휴학비교["학번"] != merge_df_기납휴학비교["휴학환불_학번"], "학번비교"] = "불일치"

# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\기납대상자와 휴학 환불자 비교 결과.xlsx"
merge_df_기납휴학비교.to_excel(save_path, index=False)
    
# 위 생성된 자료에서 pivot table 로 분석 - 학번비교 불일치 일치 확인
pivot_merge_df_기납휴학비교 = merge_df_기납휴학비교.pivot_table("기납구분",
                                                                index = ["학번비교"],
                                                                columns = "기납년도",
                                                                fill_value=0,
                                                                aggfunc = "count",
                                                                margins=True,
                                                                margins_name="계")

# 해당 값 보기
merge_df_기납휴학비교[merge_df_기납휴학비교["학번비교"] == "일치"]

# pivot_merge_df_기납휴학비교

# merge_df_기납휴학비교


,연번,학번,성명,전공,기납시_학적변동내역,기납시_학적변동일자,기납구분,기납년도,기납학기,등록구분,...,휴학환불_학과명,휴학환불_학년,휴학환불_구분,휴학환불_입학금,휴학환불_수업료,휴학환불_합계,휴학환불_처리일자,휴학환불_년도/학기,휴학환불_사유,학번비교
723,724.0,1.000000e+10,홍길동,건축학부,NaN,2022-02-26,정상기납,2022.0,1.0,미등록,...,홍길동,1.0,휴학,0.0,2268330.0,2268330.0,2021-03-11,2021/1,"21-1학기 휴학신청자 등록금 환불, 등록금 2,722,000원 5/6액(21-1학...",일치


## 등록대상자생성(재학생) 후 데이터 프레임으로 전환 및 피벗 후 학사관리팀 자료와 비교

In [8]:
# 파일 경로 및 확장자 지정 .xlsx 와 .csv에 따라 진행
file_format = ".xlsx" # .csv
file_path = "D:\C-myPyExcel\재학생 등록자료 검토" # 경로 입력
file_list = glob.glob(f"{file_path}/*{file_format}")
file_list

['D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 기납대상자 생성 결과 723명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록 후 휴학환불자 18명(21년 3월 1일부터 23년 2월 3일까지).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자생성(재학생) 등록발부자 11196명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자생성(휴학생) - 실행 후 등록발부자 2248명-가상계좌 부여 후.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자출력 8201명(재입학+재학생_학사학위취득유예+복학전액+복학제로).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 학사학위취득유예자 등록금일괄생성 등록발부자 14명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230626_(자료) 종헙정보시스템 학사행정_등록_기준정보_기준정보 중 등록금 책정 관리(전공별) - 계약학과 수정 후 검토용.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\기납대상자와 휴학 환불자 비교 결과.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_가상계좌중복.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교_피벗.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록

In [9]:
# 등록대상자생성결과(재학생) 파일 읽어오기
df_register_in_shool = pd.read_excel(
                                "230203_(자료) 종합정보 등록대상자생성(재학생) 등록발부자 11196명.xlsx",
                                usecols=["발부일자", "등록구분", "학부(과)", "학년", "학번", "입학금", "수업료", "등록금", "장학1", "장학2", "입학감면", "수업감면", "전체감면", "수납금액", "가상계좌", "수납여부"],
                                sheet_name="Sheet",
                                skipfooter=1
                                           )

# 등록구분에 따라 등록구분 그룹화 약명 추가
map_dictionary ={
                    "복학전액등록":"복학전액",
                    "복학제로등록":"복학제로",
                    "재수졸업인증수강등록":"졸업인증수강",
                    "재학생등록":"재학",
                    "편입학등록":"편입",
                    "신입학등록":"신입",
                    "학사학위취득유예등록":"학사학위취득유예",
                    "재수재입학":"재입학", "재수재입학반액등록":"재입학", "재수재입학전액등록":"재입학", "재입학전액등록":"재입학",
                    "재수1/3":"수업연한초과", "재수1/6":"수업연한초과", "재수반액등록":"수업연한초과", "재수복학1/3":"수업연한초과", "재수복학1/6":"수업연한초과", "재수복학반액":"수업연한초과",
                    "재수복학전액":"수업연한초과", "재수복학제로":"수업연한초과", "재수전액등록":"수업연한초과"} 
df_register_in_shool["등록구분(그룹)"] = df_register_in_shool["등록구분"].map(map_dictionary) 


df_register_in_shool.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11197 entries, 0 to 11196
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   발부일자      11197 non-null  datetime64[ns]
 1   등록구분      11197 non-null  object        
 2   학부(과)     11197 non-null  object        
 3   학년        11197 non-null  int64         
 4   학번        11197 non-null  int64         
 5   입학금       11197 non-null  int64         
 6   수업료       11197 non-null  int64         
 7   등록금       11197 non-null  int64         
 8   장학1       5265 non-null   object        
 9   장학2       5265 non-null   object        
 10  입학감면      11197 non-null  int64         
 11  수업감면      11197 non-null  int64         
 12  전체감면      11197 non-null  int64         
 13  수납금액      11197 non-null  int64         
 14  가상계좌      9017 non-null   float64       
 15  수납여부      11197 non-null  object        
 16  등록구분(그룹)  11196 non-null  object        
dtypes: datetime6

In [10]:
#  pivot table 로 분석 - 등록구분별 피벗생성하여 학사관리팀 자료와 비교
pivot_df_register_in_shool = df_register_in_shool.pivot_table(
                                                                "발부일자",
                                                                index = ["등록구분(그룹)", "등록구분"],
                                                                columns = "수납여부",
                                                                fill_value=0,
                                                                aggfunc = "count",
                                                                margins=True,
                                                                margins_name="계"
                                                            )


# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\피벗자료_등록대상자생성(재학생).xlsx"
pivot_df_register_in_shool.to_excel(save_path, index=True)

pivot_df_register_in_shool

수납여부                 Checked  Unchecked      계
등록구분(그룹) 등록구분                                 
복학전액     복학전액등록          821        681   1502
복학제로     복학제로등록          299         57    356
수업연한초과   재수1/3            44          1     45
         재수1/6            75          0     75
         재수반액등록           27          0     27
         재수복학1/3           3          0      3
         재수복학1/6           5          0      5
         재수복학반액            2          0      2
         재수복학전액            7         12     19
         재수복학제로            3          0      3
         재수전액등록          100         18    118
신입       신입학등록          1876          0   1876
재입학      재수재입학반액등록         1          0      1
         재수재입학전액등록         2          0      2
         재입학전액등록          18          0     18
재학       재학생등록          5649        751   6400
졸업인증수강   재수졸업인증수강등록      463          4    467
편입       편입학등록           263          0    263
학사학위취득유예 학사학위취득유예등록       14          0     14
계                       9672       1524  11196

In [11]:
df_register_in_shool["학번"].groupby(df_register_in_shool["등록구분(그룹)"]).count()

등록구분(그룹)
복학전액        1502
복학제로         356
수업연한초과       297
신입          1876
재입학           21
재학          6400
졸업인증수강       467
편입           263
학사학위취득유예      14
Name: 학번, dtype: int64

## 학사학위취득유예자 등록금일괄 생성 후 생성자료와 학사관리팀 학사학위취득유예자 명단 비교

In [12]:
# 파일 경로 및 확장자 지정 .xlsx 와 .csv에 따라 진행
file_format = ".xlsx" # .csv
file_path = "D:\C-myPyExcel\재학생 등록자료 검토" # 경로 입력
file_list = glob.glob(f"{file_path}/*{file_format}")
file_list

['D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 기납대상자 생성 결과 723명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록 후 휴학환불자 18명(21년 3월 1일부터 23년 2월 3일까지).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자생성(재학생) 등록발부자 11196명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자생성(휴학생) - 실행 후 등록발부자 2248명-가상계좌 부여 후.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자출력 8201명(재입학+재학생_학사학위취득유예+복학전액+복학제로).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 학사학위취득유예자 등록금일괄생성 등록발부자 14명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230626_(자료) 종헙정보시스템 학사행정_등록_기준정보_기준정보 중 등록금 책정 관리(전공별) - 계약학과 수정 후 검토용.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\기납대상자와 휴학 환불자 비교 결과.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_가상계좌중복.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교_피벗.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록

In [13]:
# 등록대상자결과(학사학위취득유예) 파일 읽어오기
df_register_reprieve = pd.read_excel(
                                "230203_(자료) 종합정보 학사학위취득유예자 등록금일괄생성 등록발부자 14명.xlsx",
                                usecols=["등록구분", "학부(과)", "학년", "학번", "신청학점", "학적상태", "입학금", "수업료", "등록금", "장학1", "장학2", "입학금감면", "수업료감면", "전체감면", "수납금액", "수납여부"],
                                sheet_name="Sheet",
                                skipfooter=1
                                           )

df_register_reprieve

,등록구분,학부(과),학년,학번,신청학점,학적상태,입학금,수업료,등록금,장학1,장학2,입학금감면,수업료감면,전체감면,수납금액,수납여부
0,학사학위취득유예등록,경영학부,4,2017111271,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
1,학사학위취득유예등록,기계공학부,4,2017112075,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
2,학사학위취득유예등록,기계공학부,4,2017112221,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
3,학사학위취득유예등록,전자공학과,4,2017112379,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
4,학사학위취득유예등록,컴퓨터공학부,4,2017112471,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
5,학사학위취득유예등록,건설시스템공학과,4,2017112700,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
6,학사학위취득유예등록,전자공학과,4,2017113089,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
7,학사학위취득유예등록,외식프랜차이즈학과,4,2018110747,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
8,학사학위취득유예등록,경찰학부,4,2018111837,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked
9,학사학위취득유예등록,컴퓨터공학부,4,2018112520,0,재학,0,0,0,NaN,NaN,0,0,0,0,Unchecked


In [14]:
# 학사관리팀 학사학위취득유예 파일 읽어오기
df_college_register_reprieve = pd.read_excel(
                                "붙임4 2022 전기 학사학위취득유예자 명부.xlsx",
                                usecols=["학부(과)", "학번", "졸업유예\n차수", "사유"],
                                sheet_name="Sheet",
                                skiprows=2,
                                           )

df_college_register_reprieve.rename(
                   columns={
                               "학부(과)":"학부(과)_학사",
                               "졸업유예\n차수":"졸업유예차수"
                           }
                            , inplace=True
                            )

df_college_register_reprieve

,학부(과)_학사,학번,졸업유예차수,사유
0,건설시스템공학과,2017112700,1,취업준비
1,경영학부,2017111271,1,취업준비
2,경찰학부,2018111837,1,취업준비
3,기계공학부,2017112075,1,취업준비
4,기계공학부,2017112221,1,취업준비
5,바이오융합학부,2019110403,1,취업준비
6,외식프랜차이즈학과,2018110747,1,취업준비
7,유아교육과,2018113026,1,취업준비
8,전자공학과,2017112379,1,취업준비
9,전자공학과,2017113089,1,취업준비


In [15]:
# 등록대상자결과(학사학위취득유예) 파일과 학사관리팀 학사학위취득유예 파일을 등록대상자결과(학사학귀취득유예)의 학번 기준으로 하나의 dataframe 생성됨
merge_df_학사학위취득유예비교 = pd.merge(df_register_reprieve, df_college_register_reprieve, left_on="학번", right_on="학번", how="outer")


# 인덱스 초기화
merge_df_학사학위취득유예비교.reset_index(drop=True, inplace=True)


# 학부(과) 자료 비교로 이상여부 확인
merge_df_학사학위취득유예비교.loc[merge_df_학사학위취득유예비교["학부(과)"] == merge_df_학사학위취득유예비교["학부(과)_학사"], "학과비교"] = "일치"
merge_df_학사학위취득유예비교.loc[merge_df_학사학위취득유예비교["학부(과)"] != merge_df_학사학위취득유예비교["학부(과)_학사"], "학과비교"] = "불일치"


# 등록구분에 nan 값이 존재할 경우 "이상자료"로 표시
merge_df_학사학위취득유예비교["등록구분"] = merge_df_학사학위취득유예비교["등록구분"].fillna("이상자료")
merge_df_학사학위취득유예비교["수납여부"] = merge_df_학사학위취득유예비교["수납여부"].fillna("이상자료")
merge_df_학사학위취득유예비교["입학금"] = merge_df_학사학위취득유예비교["입학금"].fillna("이상자료")
merge_df_학사학위취득유예비교["수업료"] = merge_df_학사학위취득유예비교["수업료"].fillna("이상자료")
merge_df_학사학위취득유예비교["등록금"] = merge_df_학사학위취득유예비교["등록금"].fillna("이상자료")
merge_df_학사학위취득유예비교["전체감면"] = merge_df_학사학위취득유예비교["전체감면"].fillna("이상자료")
merge_df_학사학위취득유예비교["수납금액"] = merge_df_학사학위취득유예비교["수납금액"].fillna("이상자료")

# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\학사학위취득유예자 통합 자료.xlsx"
merge_df_학사학위취득유예비교.to_excel(save_path, index=False)


# 위 생성된 자료에서 pivot table 로 분석 - 학번비교 불일치 일치 확인
pivot_merge_df_학사학위취득유예비교 = merge_df_학사학위취득유예비교.pivot_table("수납여부",
                                                                index = ["등록구분"],
                                                                columns = "학과비교",
                                                                fill_value=0,
                                                                aggfunc = "count",
                                                                margins=True,
                                                                margins_name="계")

# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\학사학위취득유예자 통합 비교 결과.xlsx"
pivot_merge_df_학사학위취득유예비교.to_excel(save_path, index=True)


pivot_merge_df_학사학위취득유예비교

학과비교,불일치,일치,계
등록구분,,,
이상자료,1,0,1
학사학위취득유예등록,1,14,15
계,2,14,16


In [16]:
# 위 생성된 자료에서 pivot table 로 분석 - 등록금과 전체감면액이 0원 인지
pivot_merge_df_학사학위취득유예비교 = merge_df_학사학위취득유예비교.pivot_table("수납금액",
                                                                index = ["등록금"],
                                                                columns = "전체감면",
                                                                fill_value=0,
                                                                aggfunc = "count",
                                                                margins=True,
                                                                margins_name="계")

# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\학사학위취득유예자 금액 비교 결과.xlsx"
pivot_merge_df_학사학위취득유예비교.to_excel(save_path, index=True)


pivot_merge_df_학사학위취득유예비교

전체감면,0.0,이상자료,계
등록금,,,
0.0,15,0,15
이상자료,0,1,1
계,15,1,16


## 재학생 및 휴학생 가상계좌 부여 후 중복 확인

In [17]:
# 파일 경로 및 확장자 지정 .xlsx 와 .csv에 따라 진행
file_format = ".xlsx" # .csv
file_path = "D:\C-myPyExcel\재학생 등록자료 검토" # 경로 입력
file_list = glob.glob(f"{file_path}/*{file_format}")
file_list

['D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 기납대상자 생성 결과 723명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록 후 휴학환불자 18명(21년 3월 1일부터 23년 2월 3일까지).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자생성(재학생) 등록발부자 11196명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자생성(휴학생) - 실행 후 등록발부자 2248명-가상계좌 부여 후.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자출력 8201명(재입학+재학생_학사학위취득유예+복학전액+복학제로).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 학사학위취득유예자 등록금일괄생성 등록발부자 14명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230626_(자료) 종헙정보시스템 학사행정_등록_기준정보_기준정보 중 등록금 책정 관리(전공별) - 계약학과 수정 후 검토용.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\기납대상자와 휴학 환불자 비교 결과.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_가상계좌중복.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교_피벗.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록

In [18]:
# 등록대상자생성결과(재학생) 파일 읽어오기
df_register_in_shool = pd.read_excel(
                                "230203_(자료) 종합정보 등록대상자생성(재학생) 등록발부자 11196명.xlsx",
                                usecols=["발부일자", "등록구분", "학부(과)", "학년", "학번", "입학금", "수업료", "등록금", "장학1", "장학2", "입학감면", "수업감면", "전체감면", "수납금액", "가상계좌", "수납여부"],
                                sheet_name="Sheet",
                                skipfooter=1
                                           )

# 등록구분에 따라 등록구분 그룹화 약명 추가
map_dictionary ={
                    "복학전액등록":"복학전액",
                    "복학제로등록":"복학제로",
                    "재수졸업인증수강등록":"졸업인증수강",
                    "재학생등록":"재학",
                    "편입학등록":"편입",
                    "신입학등록":"신입",
                    "학사학위취득유예등록":"학사학위취득유예",
                    "재수재입학":"재입학", "재수재입학반액등록":"재입학", "재수재입학전액등록":"재입학", "재입학전액등록":"재입학",
                    "재수1/3":"수업연한초과", "재수1/6":"수업연한초과", "재수반액등록":"수업연한초과", "재수복학1/3":"수업연한초과", "재수복학1/6":"수업연한초과", "재수복학반액":"수업연한초과",
                    "재수복학전액":"수업연한초과", "재수복학제로":"수업연한초과", "재수전액등록":"수업연한초과"} 
df_register_in_shool["등록구분(그룹)"] = df_register_in_shool["등록구분"].map(map_dictionary) 

# nan 값 없애기
df_register_in_shool = df_register_in_shool.fillna("")

# 등록구분 추가
df_register_in_shool["등록구분(재휴)"] = "재학"

# 데이터 타입 변경
df_register_in_shool["가상계좌"] = pd.to_numeric(df_register_in_shool["가상계좌"], errors="coerce")



df_register_in_shool.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11197 entries, 0 to 11196
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   발부일자      11197 non-null  datetime64[ns]
 1   등록구분      11197 non-null  object        
 2   학부(과)     11197 non-null  object        
 3   학년        11197 non-null  int64         
 4   학번        11197 non-null  int64         
 5   입학금       11197 non-null  int64         
 6   수업료       11197 non-null  int64         
 7   등록금       11197 non-null  int64         
 8   장학1       11197 non-null  object        
 9   장학2       11197 non-null  object        
 10  입학감면      11197 non-null  int64         
 11  수업감면      11197 non-null  int64         
 12  전체감면      11197 non-null  int64         
 13  수납금액      11197 non-null  int64         
 14  가상계좌      9017 non-null   float64       
 15  수납여부      11197 non-null  object        
 16  등록구분(그룹)  11197 non-null  object        
 17  등록구분(재휴)  11

In [19]:
# 등록대상자생성결과(휴학생) 파일 읽어오기
df_register_leave = pd.read_excel(
                                "230203_(자료) 종합정보 등록대상자생성(휴학생) - 실행 후 등록발부자 2248명-가상계좌 부여 후.xlsx",
                                usecols=["발부일자", "등록구분", "학부(과)", "학년", "학번", "입학금", "수업료", "등록금", "장학", "입학감면", "수업감면", "감면합계", "가상계좌"],
                                sheet_name="Sheet",
                                skipfooter=1
                                           )

# 등록구분에 따라 등록구분 그룹화 약명 추가
map_dictionary ={
                    "복학전액등록":"복학전액",
                    "복학제로등록":"복학제로",
                    "재수졸업인증수강등록":"졸업인증수강",
                    "재학생등록":"재학",
                    "편입학등록":"편입",
                    "신입학등록":"신입",
                    "학사학위취득유예등록":"학사학위취득유예",
                    "재수재입학":"재입학", "재수재입학반액등록":"재입학", "재수재입학전액등록":"재입학", "재입학전액등록":"재입학",
                    "재수1/3":"수업연한초과", "재수1/6":"수업연한초과", "재수반액등록":"수업연한초과", "재수복학1/3":"수업연한초과", "재수복학1/6":"수업연한초과", "재수복학반액":"수업연한초과",
                    "재수복학전액":"수업연한초과", "재수복학제로":"수업연한초과", "재수전액등록":"수업연한초과"} 
df_register_leave["등록구분(그룹)"] = df_register_leave["등록구분"].map(map_dictionary) 


df_register_leave.rename(
                   columns={
                               "감면합계":"전체감면",
                           }
                            , inplace=True
                            )

# 등록구분 추가
df_register_leave["등록구분(재휴)"] = "휴학"


# nan 값 없애기
df_register_leave = df_register_leave.fillna("")

df_register_leave.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2248 entries, 0 to 2247
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   발부일자      2248 non-null   datetime64[ns]
 1   등록구분      2248 non-null   object        
 2   학부(과)     2248 non-null   object        
 3   학년        2248 non-null   int64         
 4   학번        2248 non-null   int64         
 5   입학금       2248 non-null   int64         
 6   수업료       2248 non-null   int64         
 7   등록금       2248 non-null   int64         
 8   장학        2248 non-null   object        
 9   입학감면      2248 non-null   int64         
 10  수업감면      2248 non-null   int64         
 11  전체감면      2248 non-null   int64         
 12  가상계좌      2248 non-null   int64         
 13  등록구분(그룹)  2248 non-null   object        
 14  등록구분(재휴)  2248 non-null   object        
dtypes: datetime64[ns](1), int64(9), object(5)
memory usage: 263.6+ KB


In [20]:
# 위 등록대상자생성결과(재학생 + 휴학생)를 하나의 dataframe 생성됨 -아래로 병합: concat
co_df_register_in_school_leave = pd.concat([df_register_in_shool, df_register_leave], ignore_index=True)

co_df_register_in_school_leave["장학_통합"] = co_df_register_in_school_leave["장학1"].map(str) + ", " + co_df_register_in_school_leave["장학2"].map(str) + ", " + co_df_register_in_school_leave["장학"].map(str)

co_df_register_in_school_leave = co_df_register_in_school_leave.drop(columns=["장학1", "장학2", "장학"], axis=1)


# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\등록대상자생성결과(재학생 + 휴학생).xlsx"
co_df_register_in_school_leave.to_excel(save_path, index=True)

co_df_register_in_school_leave.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13445 entries, 0 to 13444
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   발부일자      13445 non-null  datetime64[ns]
 1   등록구분      13445 non-null  object        
 2   학부(과)     13445 non-null  object        
 3   학년        13445 non-null  int64         
 4   학번        13445 non-null  int64         
 5   입학금       13445 non-null  int64         
 6   수업료       13445 non-null  int64         
 7   등록금       13445 non-null  int64         
 8   입학감면      13445 non-null  int64         
 9   수업감면      13445 non-null  int64         
 10  전체감면      13445 non-null  int64         
 11  수납금액      11197 non-null  float64       
 12  가상계좌      11265 non-null  float64       
 13  수납여부      11197 non-null  object        
 14  등록구분(그룹)  13445 non-null  object        
 15  등록구분(재휴)  13445 non-null  object        
 16  장학_통합     13445 non-null  object        
dtypes: datetime6

In [21]:
# 위 등록대상자생성결과(재학생 + 휴학생)에서 가상계좌 기준으로 합쳐진 중복 값 생성
merge_register_in_school_leave_가상 = pd.merge(df_register_in_shool, df_register_leave, how="inner", left_on="가상계좌", right_on="가상계좌", suffixes=('_재학', '_휴학'))

# 자료 비교로 이상여부 확인
merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["등록구분_재학"] == merge_register_in_school_leave_가상["등록구분_휴학"], "등록구분비교"] = "일치"
merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["등록구분_재학"] != merge_register_in_school_leave_가상["등록구분_휴학"], "등록구분비교"] = "일치"

merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["학년_재학"] == merge_register_in_school_leave_가상["학년_휴학"], "학년비교"] = "일치"
merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["학년_재학"] != merge_register_in_school_leave_가상["학년_휴학"], "학년비교"] = "불일치"

merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["학번_재학"] == merge_register_in_school_leave_가상["학번_휴학"], "학번비교"] = "일치"
merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["학번_재학"] != merge_register_in_school_leave_가상["학번_휴학"], "학번비교"] = "불일치"

merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["등록금_재학"] == merge_register_in_school_leave_가상["등록금_휴학"], "등록금비교"] = "일치"
merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["등록금_재학"] != merge_register_in_school_leave_가상["등록금_휴학"], "등록금비교"] = "불일치"

merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["전체감면_재학"] == merge_register_in_school_leave_가상["전체감면_휴학"], "전체감면비교"] = "일치"
merge_register_in_school_leave_가상.loc[merge_register_in_school_leave_가상["전체감면_재학"] != merge_register_in_school_leave_가상["전체감면_휴학"], "전체감면비교"] = "불일치"

merge_register_in_school_leave_가상 = merge_register_in_school_leave_가상[["발부일자_재학", "학번_재학", "학번비교", "등록구분_재학", "등록구분비교", "학년_재학", "학년비교", "등록금_재학", "등록금비교", "전체감면_재학", "전체감면비교", "수납금액", "가상계좌", "발부일자_휴학"]]


# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\등록대상자생성결과(재학생 + 휴학생)_가상계좌중복.xlsx"
merge_register_in_school_leave_가상.to_excel(save_path, index=True)

merge_register_in_school_leave_가상.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 88
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   발부일자_재학  89 non-null     datetime64[ns]
 1   학번_재학    89 non-null     int64         
 2   학번비교     89 non-null     object        
 3   등록구분_재학  89 non-null     object        
 4   등록구분비교   89 non-null     object        
 5   학년_재학    89 non-null     int64         
 6   학년비교     89 non-null     object        
 7   등록금_재학   89 non-null     int64         
 8   등록금비교    89 non-null     object        
 9   전체감면_재학  89 non-null     int64         
 10  전체감면비교   89 non-null     object        
 11  수납금액     89 non-null     int64         
 12  가상계좌     89 non-null     float64       
 13  발부일자_휴학  89 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(6)
memory usage: 10.4+ KB


In [22]:
# 위 등록대상자생성결과(재학생 + 휴학생)를 하나의 dataframe 생성됨 -아래로 병합: concat - 재학과 휴학에 중복된 자료도 각각 검증
co_register_등록금 = pd.concat([df_register_in_shool, df_register_leave], ignore_index=True)

co_register_등록금 = co_register_등록금[[
                                            "발부일자",
                                            "등록구분",   
                                            "학부(과)",
                                            "학년",
                                            "학번",
                                            "입학금",
                                            "수업료",
                                            "등록금",
                                            "입학감면",
                                            "수업감면",
                                            "전체감면",
                                            "등록구분(그룹)",
                                            "등록구분(재휴)"
                                        ]]

co_register_등록금["입학금+수업료"] = co_register_등록금["입학금"] + co_register_등록금["수업료"]
co_register_등록금["입학감면+수업감면"] = co_register_등록금["입학감면"] + co_register_등록금["수업감면"]

# 책정 등록금 = 등록금_재학 + 전체감면_재학
co_register_등록금["책정 등록금"] = co_register_등록금["등록금"] + co_register_등록금["전체감면"]


# 자료 비교로 이상여부 확인
co_register_등록금.loc[co_register_등록금["등록금"] == co_register_등록금["입학금+수업료"], "등록금비교"] = "일치"
co_register_등록금.loc[co_register_등록금["등록금"] != co_register_등록금["입학금+수업료"], "등록금비교"] = "불일치"

co_register_등록금.loc[co_register_등록금["전체감면"] == co_register_등록금["입학감면+수업감면"], "전체감면비교"] = "일치"
co_register_등록금.loc[co_register_등록금["전체감면"] != co_register_등록금["입학감면+수업감면"], "전체감면비교"] = "불일치"

# 위 생성된 자료에서 pivot table 로 분석 - 
pivot_co_register_등록금 = co_register_등록금.pivot_table("학번",
                                                          index = ["등록구분(재휴)"],
                                                          columns = ["등록금비교", "전체감면비교"],
                                                          fill_value=0,
                                                          aggfunc = "count",
                                                          margins=True,
                                                          margins_name="계")


# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\등록대상자생성결과(재학생 + 휴학생)_책정등록금확인.xlsx"
co_register_등록금.to_excel(save_path, index=True)

pivot_co_register_등록금

등록금비교,불일치,일치,계
전체감면비교,불일치,일치,
등록구분(재휴),,,
재학,1,11196,11197
휴학,0,2248,2248
계,1,13444,13445


In [23]:
# 학과별 등록금 책정 자료 읽어오기
df_register_table = pd.read_excel(
                                "230626_(자료) 종헙정보시스템 학사행정_등록_기준정보_기준정보 중 등록금 책정 관리(전공별) - 계약학과 수정 후 검토용.xlsx",
                                sheet_name="피벗결과"
                                           )

df_register_table

,학부(과),단과대학,계열구분,사용여부,입학금,입학금 실비용,수업료,총합계
0,가정교육과,사범대학,이학계열,Checked,0,112400,3211000,3323400
1,간호학과,건강보건대학,이학계열,Checked,0,112400,3211000,3323400
2,건축학부,공과대학,공학계열,Checked,0,112400,3750000,3862400
3,경영학과,경영대학,"인문,사회",Checked,0,112400,2200000,2312400
4,경영학부,경영대학,"인문,사회",Checked,0,112400,2722000,2834400
...,...,...,...,...,...,...,...,...
69,토목공학과,공과대학,공학계열,Checked,0,112400,3750000,3862400
70,산업디자인학과,공과대학,예능,Checked,0,112400,3869000,3981400
71,식품영양생명학부,건강보건대학,이학계열,Checked,0,112400,3211000,3323400
72,국제무역물류학과,경영대학,"인문,사회",Checked,0,112400,2722000,2834400


In [24]:
# 위 등록대상자생성결과(재학생 + 휴학생)에서 가상계좌 기준으로 합쳐진 중복 값 생성
merge_register_table = pd.merge(co_register_등록금, df_register_table, how="outer", left_on="학부(과)", right_on="학부(과)", suffixes=('_책정', '_table'))


# 위 생성된 자료에서 pivot table 로 분석 - 
pivot_merge_register_table = merge_register_table.pivot_table("학번",
                                                          index = ["계열구분", "학부(과)", "등록구분(그룹)", "책정 등록금", "입학금 실비용", "수업료_table"],
                                                          columns = ["등록구분(재휴)"],
                                                          fill_value=0,
                                                          aggfunc = "count",
                                                          margins=True,
                                                          margins_name="계")

# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교.xlsx"
merge_register_table.to_excel(save_path, index=True)

# merge_register_table

# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교_피벗.xlsx"
pivot_merge_register_table.to_excel(save_path, index=True)

## 등록대상자생성결과(재학생 + 휴학생)에서 전과자 등록금 확인

In [34]:
# 파일 경로 및 확장자 지정 .xlsx 와 .csv에 따라 진행
file_format = ".xlsx" # .csv
file_path = "D:\C-myPyExcel\재학생 등록자료 검토" # 경로 입력
file_list = glob.glob(f"{file_path}/*{file_format}")
file_list

['D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 기납대상자 생성 결과 723명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록 후 휴학환불자 18명(21년 3월 1일부터 23년 2월 3일까지).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자생성(재학생) 등록발부자 11196명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자생성(휴학생) - 실행 후 등록발부자 2248명-가상계좌 부여 후.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 등록대상자출력 8201명(재입학+재학생_학사학위취득유예+복학전액+복학제로).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230203_(자료) 종합정보 학사학위취득유예자 등록금일괄생성 등록발부자 14명.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\230626_(자료) 종헙정보시스템 학사행정_등록_기준정보_기준정보 중 등록금 책정 관리(전공별) - 계약학과 수정 후 검토용.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\~$붙임8-2 2023-1 특별전과 합격자(학과용).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\기납대상자와 휴학 환불자 비교 결과.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생).xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_가상계좌중복.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자생성결과(재학생 + 휴학생)_책정등록금비교.xlsx',
 'D:\\C-myPyExcel\\재학생 등록자료 검토\\등록대상자

In [78]:
# 일반 전과 합격자 파일 읽어오기
df_일전 = pd.read_excel(
                        "붙임8-1 2023-1 전과합격자(학과용).xlsx",
                        usecols=["학번", "학년", "학적\n상태", "전출학부(과)", "전입학(부)과", "전출입\n학년"],
                        sheet_name="Sheet"
                        )

# 명칭 변경
df_일전.rename(
                   columns={
                                "학적\n상태":"학적상태",
                                "학년":"전출학년",
                                "전출입\n학년":"전입학년",
                                "전출학부(과)":"전출학과",
                                "전입학(부)과":"전입학과"
                           },
                   inplace=True
                )

# 구분 추가
df_일전["전과구분"] = "일반"

df_일전.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   학번      59 non-null     int64 
 1   전출학년    59 non-null     int64 
 2   학적상태    59 non-null     object
 3   전출학과    59 non-null     object
 4   전입학과    59 non-null     object
 5   전입학년    59 non-null     int64 
 6   전과구분    59 non-null     object
dtypes: int64(3), object(4)
memory usage: 3.4+ KB


In [79]:
# 특별 전과 합격자 파일 읽어오기
df_특전 = pd.read_excel(
                        "붙임8-2 2023-1 특별전과 합격자(학과용).xlsx",
                        usecols=["학번", "학년", "학적\n상태", "현 소속", "전부(과) 소속", "전부(과)\n학년", "복학예정연도", "복학예정학기"],
                        skiprows=1,
                        sheet_name="Sheet"
                        )

# 명칭 변경
df_특전.rename(
                   columns={
                               "학적\n상태":"학적상태",
                               "학년":"전출학년",
                               "현 소속":"전출학과",
                               "전부(과) 소속":"전입학과",
                               "전부(과)\n학년":"전입학년"
                           },
                   inplace=True
                )

# 구분 추가
df_특전["전과구분"] = "특별"

# nan 값 없애기
df_특전["복학예정연도"] = df_특전["복학예정연도"].fillna("재학")
df_특전["복학예정학기"] = df_특전["복학예정학기"].fillna("재학")


df_특전.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   학번      19 non-null     int64 
 1   전출학년    19 non-null     int64 
 2   학적상태    19 non-null     object
 3   전출학과    19 non-null     object
 4   전입학과    19 non-null     object
 5   전입학년    19 non-null     int64 
 6   복학예정연도  19 non-null     object
 7   복학예정학기  19 non-null     object
 8   전과구분    19 non-null     object
dtypes: int64(3), object(6)
memory usage: 1.5+ KB


In [80]:
# 위 일전 + 특전 에서 학번 기준으로 합쳐진 중복 값 생성
co_일전_특전 = pd.concat([df_일전, df_특전], ignore_index=True)

co_일전_특전

,학번,전출학년,학적상태,전출학과,전입학과,전입학년,전과구분,복학예정연도,복학예정학기
0,2022110250,1,재학,스포츠과학과,건설시스템공학과,2,일반,NaN,NaN
1,2019111867,1,재학,경찰학부,건설시스템공학과,2,일반,NaN,NaN
2,2018112860,3,휴학,조선해양시스템공학과,건설시스템공학과,3,일반,NaN,NaN
3,2019112765,2,재학,환경에너지공학과,건축학부,3,일반,NaN,NaN
4,2018112863,3,재학,조선해양시스템공학과,건축학부,4,일반,NaN,NaN
...,...,...,...,...,...,...,...,...,...
73,2021110051,2,휴학,중국학과,미디어영상학과,2,특별,2023.0,1.0
74,2020110028,2,휴학,한국어문학과,건설시스템공학과,2,특별,2023.0,1.0
75,2022111573,1,재학,환경에너지공학과,전자공학과,2,특별,재학,재학
76,2020112611,2,휴학,환경에너지공학과,컴퓨터공학부,2,특별,2023.0,1.0


In [93]:
# 위 자료에서 중복 학번 있는지 확인
co_일전_특전_중복 = co_일전_특전[co_일전_특전.duplicated(subset="학번", keep=False)]

co_일전_특전_중복

,학번,전출학년,학적상태,전출학과,전입학과,전입학년,전과구분,복학예정연도,복학예정학기
58,2018112061,2,재학,기계공학부,행정학과,3,일반,NaN,NaN
77,2018112061,2,재학,기계공학부,행정학과,2,특별,2023.0,1.0


In [97]:
# 위 자료에서 중복 학번 있을 경우 제거
# 중복 데이터 전부 제거 :: False, df.drop_duplicates(['MF', 'Age'], keep = False)
# 맨 위 / 첫 번째 행 남기고 전부 제거 :: first, df.drop_duplicates(['MF', 'Age'], keep = 'first')
# 맨 아래 / 마지막 행 남기고 전부 제거 :: last, df.drop_duplicates(['MF', 'Age'], keep = 'last')
# keep 에 아무것도 지정하지 않으면 디폴트 값은 'first'

co_일전_특전_중복제거 = co_일전_특전.drop_duplicates(subset="학번", inplace=False, ignore_index=False)

#co_일전_특전_중복 = co_일전_특전[co_일전_특전.drop_duplicates(subset="학번", inplace=False, ignore_index=False)]

co_일전_특전_중복제거

,학번,전출학년,학적상태,전출학과,전입학과,전입학년,전과구분,복학예정연도,복학예정학기
0,2022110250,1,재학,스포츠과학과,건설시스템공학과,2,일반,NaN,NaN
1,2019111867,1,재학,경찰학부,건설시스템공학과,2,일반,NaN,NaN
2,2018112860,3,휴학,조선해양시스템공학과,건설시스템공학과,3,일반,NaN,NaN
3,2019112765,2,재학,환경에너지공학과,건축학부,3,일반,NaN,NaN
4,2018112863,3,재학,조선해양시스템공학과,건축학부,4,일반,NaN,NaN
...,...,...,...,...,...,...,...,...,...
72,2022110025,1,재학,중국학과,문화콘텐츠학과,2,특별,재학,재학
73,2021110051,2,휴학,중국학과,미디어영상학과,2,특별,2023.0,1.0
74,2020110028,2,휴학,한국어문학과,건설시스템공학과,2,특별,2023.0,1.0
75,2022111573,1,재학,환경에너지공학과,전자공학과,2,특별,재학,재학


In [70]:
# 위 일전 + 특전 에서 학번 기준으로 합쳐진 중복 값 생성
co_일전_특전 = pd.concat([df_일전, df_특전], ignore_index=True)
# merge_일전_특전 = pd.merge(, how="outer", left_on="학번", right_on="학번", suffixes=('_일전', '_특전'))

# nan 값 없애기
co_일전_특전["복학예정연도"] = co_일전_특전["복학예정연도"].fillna("없음")
co_일전_특전["복학예정학기"] = co_일전_특전["복학예정학기"].fillna("없음")

# 위 생성된 자료에서 pivot table 로 분석 - 
pivot_co_일전_특전 = co_일전_특전.pivot_table(
                                                    "학번",
                                                    index = ["전과구분"],
                                                    columns = ["전입학년"],
                                                    fill_value=0,
                                                    aggfunc = "count",
                                                    margins=True,
                                                    margins_name="계")
pivot_co_일전_특전


전입학년,2,3,4,계
전과구분,,,,
일반,29,22,8,59
특별,19,0,0,19
계,48,22,8,78


In [65]:
# 위 자료에서 등록자료 붙이기
merge_일전_특전_등록자료 = pd.merge(co_일전_특전, merge_register_table, how="inner", left_on="학번", right_on="학번", suffixes=('_전과', '_등록'))


# 엑셀 파일 저장
save_path = "D:\C-myPyExcel\재학생 등록자료 검토\전과자_등록대상자생성결과(재학생 + 휴학생)_비교.xlsx"
merge_일전_특전_등록자료.to_excel(save_path, index=True) 

merge_일전_특전_등록자료

,학번,전출학년,학적상태,전출학과,전입학과,전입학년,전과구분,복학예정연도,복학예정학기,발부일자,...,책정 등록금,등록금비교,전체감면비교,단과대학,계열구분,사용여부,입학금_table,입학금 실비용,수업료_table,총합계
0,2022110250,1,재학,스포츠과학과,건설시스템공학과,2,일반,없음,없음,2023-02-03 11:30:17,...,3750000,일치,일치,공과대학,공학계열,Checked,0,112400,3750000,3862400
1,2019111867,1,재학,경찰학부,건설시스템공학과,2,일반,없음,없음,2023-02-03 11:30:17,...,3750000,일치,일치,공과대학,공학계열,Checked,0,112400,3750000,3862400
2,2018112860,3,휴학,조선해양시스템공학과,건설시스템공학과,3,일반,없음,없음,2023-02-03 11:30:17,...,3750000,일치,일치,공과대학,공학계열,Checked,0,112400,3750000,3862400
3,2019112765,2,재학,환경에너지공학과,건축학부,3,일반,없음,없음,2023-02-03 11:30:17,...,3750000,일치,일치,공과대학,공학계열,Checked,0,112400,3750000,3862400
4,2018112863,3,재학,조선해양시스템공학과,건축학부,4,일반,없음,없음,2023-02-03 11:30:17,...,3750000,일치,일치,공과대학,공학계열,Checked,0,112400,3750000,3862400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2022110025,1,재학,중국학과,문화콘텐츠학과,2,특별,재학,재학,2023-02-03 11:30:17,...,3211000,일치,일치,인문사회대학,"인문,공학",Checked,0,112400,3211000,3323400
76,2021110051,2,휴학,중국학과,미디어영상학과,2,특별,2023.0,1.0,2023-02-03 11:30:17,...,0,일치,일치,인문사회대학,"인문,사회",Checked,0,112400,2722000,2834400
77,2020110028,2,휴학,한국어문학과,건설시스템공학과,2,특별,2023.0,1.0,2023-02-03 11:30:17,...,3750000,일치,일치,공과대학,공학계열,Checked,0,112400,3750000,3862400
78,2022111573,1,재학,환경에너지공학과,전자공학과,2,특별,재학,재학,2023-02-03 11:30:17,...,3750000,일치,일치,AI·SW융합대학,공학계열,Checked,0,112400,3750000,3862400
